## Estrategia de Decodificación: Greedy Search (Búsqueda Voraz)

**Concepto General:**

La **Greedy Search** (o Búsqueda Voraz) es la estrategia de decodificación más simple y directa para los modelos secuencia a secuencia (Seq2Seq). En cada paso de la generación de la secuencia de salida, Greedy Search elige el token (palabra, subpalabra, o caracter) que tiene la probabilidad más alta según la predicción del modelo en ese momento, sin considerar el impacto de esta elección en los pasos futuros.


**Fórmula:**

Matemáticamente, en cada paso de tiempo $t$, Greedy Search selecciona el token $\hat{y}_t$ del vocabulario $V$ que maximiza la probabilidad condicional dada la secuencia de entrada $x$ y los tokens previamente generados $y_{<t}$:

$$
\hat{y}_t = \arg\max_{v \in V} P(y_t=v \mid y_{<t}, x)
$$

Donde:
* $\hat{y}_t$: es el token predicho en el paso de tiempo $t$.
* $v$: es un token candidato del vocabulario $V$.
* $P(y_t=v \mid y_{<t}, x)$: es la probabilidad (generalmente la salida de una capa softmax en el decodificador) del token $v$ en el tiempo $t$, dados los tokens anteriores $y_{<t}$ y la secuencia de entrada $x$.

El proceso se repite hasta que se genera un token especial de fin de secuencia (EOS, End-Of-Sequence) o se alcanza una longitud máxima predefinida.


**Ventajas:**

1.  Simpleza
2.  Rapidez
3.  Bajo coste computacional

**Desventajas:**

1.  **Óptimos Locales:** Es muy propensa a caer en óptimos locales. Una elección que parece la mejor en un paso puede llevar a una secuencia global subóptima. Por ejemplo, puede generar una palabra muy probable al principio que luego dificulte la formación de una frase coherente.
2.  **Repeticiones:** Puede generar secuencias repetitivas, especialmente si el modelo no está bien entrenado o si ciertas palabras tienen probabilidades consistentemente altas.
3.  **Falta de Diversidad:** No explora diferentes caminos o alternativas en la generación. Siempre produce la misma secuencia de salida para la misma entrada.
4.  **Sensibilidad a Errores Tempranos:** Un error cometido al principio de la secuencia (elegir una palabra incorrecta pero con alta probabilidad local) se propagará y afectará negativamente al resto de la generación, ya que no hay mecanismo de corrección o reconsideración.
5.  **Calidad de Secuencia:** A menudo produce secuencias que son gramaticalmente correctas pero pueden ser menos naturales, menos coherentes o de menor calidad general en comparación con métodos más avanzados.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k # Asegúrate que Multi30k esté correctamente instalado y accesible
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torchdata.datapipes.iter import IterableWrapper, Mapper 

import numpy as np
import random
import math
import time
from tqdm import tqdm


c:\Users\KATANA\Desktop\git\Personal\nlp-proyecto11\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Seq2Seq

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {DEVICE}")

SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

SRC_LANGUAGE = 'de'
TRG_LANGUAGE = 'en'

Usando dispositivo: cpu


In [3]:
token_transform = {}
vocab_transform = {}

token_transform = {}
vocab_transform = {}

 # Inicializar tokenizadores
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TRG_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
print("Tokenizadores cargados.")


def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TRG_LANGUAGE: 1}
    for data_sample in data_iter:
        if isinstance(data_sample, (list, tuple)) and len(data_sample) > language_index[language]:
                yield token_transform[language](data_sample[language_index[language]])



# Tokens especiales y sus índices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']


train_iter_for_vocab = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))

for ln in [SRC_LANGUAGE, TRG_LANGUAGE]:
    print(f"Construyendo vocabulario para: {ln}")
    current_train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TRG_LANGUAGE))
    vocab_transform[ln] = build_vocab_from_iterator(
                                yield_tokens(current_train_iter, ln),
                                min_freq=1, 
                                specials=special_symbols,
                                special_first=True)
    vocab_transform[ln].set_default_index(UNK_IDX)
print("Vocabularios construidos exitosamente desde Multi30k.")
print(f"Tamaño vocabulario alemán ({SRC_LANGUAGE}): {len(vocab_transform[SRC_LANGUAGE])}")
print(f"Tamaño vocabulario inglés ({TRG_LANGUAGE}): {len(vocab_transform[TRG_LANGUAGE])}")

if SRC_LANGUAGE not in vocab_transform or TRG_LANGUAGE not in vocab_transform or \
    len(vocab_transform.get(SRC_LANGUAGE, [])) <= len(special_symbols) or \
    len(vocab_transform.get(TRG_LANGUAGE, [])) <= len(special_symbols): # Chequeo más robusto


    if SRC_LANGUAGE not in vocab_transform or len(vocab_transform.get(SRC_LANGUAGE, [])) <= len(special_symbols):
        print(f"Creando vocabulario dummy para {SRC_LANGUAGE}")
        dummy_tokens_src = [['ein', 'mann', 'mit', 'einem', 'roten', 'helm'], ['eine', 'frau', 'spielt', 'ball']]
        dummy_vocab_src = build_vocab_from_iterator(dummy_tokens_src, specials=special_symbols, special_first=True)
        dummy_vocab_src.set_default_index(UNK_IDX)
        vocab_transform[SRC_LANGUAGE] = dummy_vocab_src

    if TRG_LANGUAGE not in vocab_transform or len(vocab_transform.get(TRG_LANGUAGE, [])) <= len(special_symbols):
        print(f"Creando vocabulario dummy para {TRG_LANGUAGE}")
        dummy_tokens_trg = [['a', 'man', 'with', 'a', 'red', 'helmet'], ['a', 'woman', 'plays', 'ball']]
        dummy_vocab_trg = build_vocab_from_iterator(dummy_tokens_trg, specials=special_symbols, special_first=True)
        dummy_vocab_trg.set_default_index(UNK_IDX)
        vocab_transform[TRG_LANGUAGE] = dummy_vocab_trg
    
    print(f"Tamaño vocabulario alemán (dummy): {len(vocab_transform[SRC_LANGUAGE])}")
    print(f"Tamaño vocabulario inglés (dummy): {len(vocab_transform[TRG_LANGUAGE])}")



Tokenizadores cargados.
Construyendo vocabulario para: de
Construyendo vocabulario para: en
Vocabularios construidos exitosamente desde Multi30k.
Tamaño vocabulario alemán (de): 19214
Tamaño vocabulario inglés (en): 10837


In [4]:
import torch
import torch.nn as nn
import random 


class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout if n_layers > 1 else 0)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # src = [src len, batch size]
        embedded = self.dropout(self.embedding(src))
        # embedded = [src len, batch size, emb dim]
        # Usar self.lstm aquí también
        outputs, (hidden, cell) = self.lstm(embedded)
        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout if n_layers > 1 else 0)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        # input = [batch size]
        # hidden = [n layers, batch size, hid dim]
        # cell = [n layers, batch size, hid dim]
        input = input.unsqueeze(0) # input = [1, batch size]
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch size, emb dim]
        # Usar self.lstm aquí también
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        # output = [1, batch size, hid dim]
        # hidden = [n layers, batch size, hid dim]
        # cell = [n layers, batch size, hid dim]
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch size, output dim]
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hid_dim == decoder.hid_dim, \
            "Las dimensiones ocultas del codificador y decodificador deben ser iguales!"
        assert encoder.n_layers == decoder.n_layers, \
            "El codificador y decodificador deben tener el mismo número de capas!"

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):

        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        hidden, cell = self.encoder(src)
        input_token = trg[0,:] # Token <bos>

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input_token, hidden, cell)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input_token = trg[t] if teacher_force else top1
        return outputs

In [5]:
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 256
N_LAYERS = 1
ENC_DROPOUT = 0.3 
DEC_DROPOUT = 0.3 

In [6]:
INPUT_DIM = len(vocab_transform[SRC_LANGUAGE])
OUTPUT_DIM = len(vocab_transform[TRG_LANGUAGE])

print(f"INPUT_DIM (tamaño vocabulario alemán): {INPUT_DIM}")
print(f"OUTPUT_DIM (tamaño vocabulario inglés): {OUTPUT_DIM}")

INPUT_DIM (tamaño vocabulario alemán): 19214
OUTPUT_DIM (tamaño vocabulario inglés): 10837


In [7]:
try:
    enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
    dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
    model = Seq2Seq(enc, dec, DEVICE).to(DEVICE)
    print("Instancias de Encoder, Decoder y Seq2Seq creadas.")
except NameError as ne:
    print(f"Error: Parece que las clases Encoder, Decoder o Seq2Seq no están definidas. Asegúrate de ejecutar la Celda 2. ({ne})")
    raise
except Exception as e_init:
    print(f"Error al instanciar los modelos: {e_init}")
    raise

Instancias de Encoder, Decoder y Seq2Seq creadas.


In [8]:
MODEL_PATH = 'RNN-TR-model.pt'

model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))

<All keys matched successfully>

In [9]:


INPUT_DIM = len(vocab_transform[SRC_LANGUAGE])
OUTPUT_DIM = len(vocab_transform[TRG_LANGUAGE])

print(f"INPUT_DIM (tamaño vocabulario alemán): {INPUT_DIM}")
print(f"OUTPUT_DIM (tamaño vocabulario inglés): {OUTPUT_DIM}")


ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 256
N_LAYERS = 1 
ENC_DROPOUT = 0.3 
DEC_DROPOUT = 0.3 


print(f"Usando Hiperparámetros CORREGIDOS para instanciar el modelo:")
print(f"  ENC_EMB_DIM: {ENC_EMB_DIM}, DEC_EMB_DIM: {DEC_EMB_DIM}")
print(f"  HID_DIM: {HID_DIM}, N_LAYERS: {N_LAYERS}")
print(f"  ENC_DROPOUT: {ENC_DROPOUT}, DEC_DROPOUT: {DEC_DROPOUT}")


enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec, DEVICE).to(DEVICE)

MODEL_PATH = 'RNN-TR-model.pt'

model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))    

INPUT_DIM (tamaño vocabulario alemán): 19214
OUTPUT_DIM (tamaño vocabulario inglés): 10837
Usando Hiperparámetros CORREGIDOS para instanciar el modelo:
  ENC_EMB_DIM: 128, DEC_EMB_DIM: 128
  HID_DIM: 256, N_LAYERS: 1
  ENC_DROPOUT: 0.3, DEC_DROPOUT: 0.3


<All keys matched successfully>

### Greedy Search

In [ ]:

def preprocess_sentence(sentence: str, vocab, tokenizer, device, bos_token='<bos>', eos_token='<eos>', unk_token='<unk>'):
    """
    Preprocesses a source sentence string into a tensor for the model.
    Args:
        sentence (str): The source language sentence.
        vocab: The source language vocabulary object (e.g., vocab_transform[SRC_LANGUAGE]).
        tokenizer: The source language tokenizer (e.g., token_transform[SRC_LANGUAGE]).
        device: The device (cpu/cuda) to send the tensor to.
        bos_token (str): The beginning-of-sequence token string.
        eos_token (str): The end-of-sequence token string.
        unk_token (str): The unknown token string.
    Returns:
        torch.Tensor: Tensor of the processed sentence, shape [seq_len, 1] (batch_size=1).
    """
    if not vocab or not tokenizer:
        print("Error: Vocabulario o tokenizador no proporcionado a preprocess_sentence.")
        return None

    # Tokenize the sentence and convert to lowercase
    tokens = [token.lower() for token in tokenizer(sentence)]
    
    # Add <bos> and <eos> tokens
    processed_tokens = [bos_token] + tokens + [eos_token]
    
    # Numericalize tokens, using <unk> for out-of-vocabulary words
    numericalized_tokens = []
    for token in processed_tokens:
        try:
            numericalized_tokens.append(vocab[token])
        except KeyError: # More robust than checking `if token in vocab` for some vocab types
            numericalized_tokens.append(vocab[unk_token])
            
    # Convert to a LongTensor and add a batch dimension (batch_size=1)
    src_tensor = torch.LongTensor(numericalized_tokens).unsqueeze(1).to(device)
    # src_tensor shape: [seq_len, 1]
    
    return src_tensor

In [ ]:
# Celda 5: Implementación de Greedy Search Decode (Corregida)

# Asegúrate que model, vocab_transform, token_transform, SRC_LANGUAGE, TRG_LANGUAGE, DEVICE
# y la función preprocess_sentence estén definidos de celdas anteriores.
# También, los tokens especiales como '<bos>', '<eos>' deben ser consistentes.

import torch # Asegurarse que torch está importado aquí si no lo está globalmente en la celda

def greedy_search_decode(model_to_decode, src_sentence_str: str, max_len: int = 50,
                         src_vocab=None, trg_vocab=None, src_tokenizer=None, device_to_use=None,
                         bos_token_str='<bos>', eos_token_str='<eos>', unk_token_str='<unk>'):
    """
    Genera una traducción de una oración fuente usando la estrategia de Greedy Search.

    Args:
        model_to_decode (torch.nn.Module): El modelo Seq2Seq entrenado.
        src_sentence_str (str): La oración en el idioma fuente.
        max_len (int): La longitud máxima permitida para la secuencia traducida.
        src_vocab: Vocabulario del idioma fuente.
        trg_vocab: Vocabulario del idioma objetivo.
        src_tokenizer: Tokenizador para el idioma fuente.
        device_to_use: Dispositivo (cpu/cuda) donde se ejecutan los cálculos.
        bos_token_str (str): Token de inicio de secuencia.
        eos_token_str (str): Token de fin de secuencia.
        unk_token_str (str): Token para palabras desconocidas.

    Returns:
        str: La oración traducida.
        None: Si ocurren errores críticos (ej. vocabularios no disponibles).
    """

    if src_vocab is None: src_vocab = vocab_transform.get(SRC_LANGUAGE)
    if trg_vocab is None: trg_vocab = vocab_transform.get(TRG_LANGUAGE)
    if src_tokenizer is None: src_tokenizer = token_transform.get(SRC_LANGUAGE)
    if device_to_use is None: device_to_use = DEVICE


    if not all([src_vocab, trg_vocab, src_tokenizer, device_to_use, model_to_decode]):
        print("Error en greedy_search_decode: Faltan componentes esenciales (modelo, vocabularios, tokenizador o dispositivo).")
        return None

    model_to_decode.eval()

    src_tensor = preprocess_sentence(
        src_sentence_str,
        src_vocab,
        src_tokenizer,
        device_to_use,
        bos_token=bos_token_str,
        eos_token=eos_token_str,
        unk_token=unk_token_str
    )

    with torch.no_grad():
        hidden, cell = model_to_decode.encoder(src_tensor)

    try:
        trg_bos_idx = trg_vocab[bos_token_str]
    except KeyError:
        print(f"Error: El token '{bos_token_str}' no se encuentra en el vocabulario objetivo.")
        return None
        
    trg_indexes = [trg_bos_idx]
    try:
        trg_eos_idx = trg_vocab[eos_token_str]
    except KeyError:
        print(f"Error: El token '{eos_token_str}' no se encuentra en el vocabulario objetivo.")
        trg_eos_idx = -1 # Un valor que probablemente no coincidirá

    for _ in range(max_len):
        trg_token_tensor = torch.LongTensor([trg_indexes[-1]]).to(device_to_use)

        with torch.no_grad():
            output_logits, hidden, cell = model_to_decode.decoder(trg_token_tensor, hidden, cell)
        
        predicted_token_index = output_logits.argmax(1).item()
        trg_indexes.append(predicted_token_index)

        if predicted_token_index == trg_eos_idx:
            break
            

    itos_map = trg_vocab.get_itos() 
    trg_tokens = [itos_map[i] for i in trg_indexes]

    if trg_tokens and trg_tokens[0] == bos_token_str:
        trg_tokens = trg_tokens[1:]
    if trg_tokens and trg_tokens[-1] == eos_token_str:
        trg_tokens = trg_tokens[:-1]
        
    return " ".join(trg_tokens)

In [12]:
german_sentences_to_translate = [
    "Ein Mann mit einem roten Helm, der auf einem Pferd reitet.",
    "Eine Gruppe von Menschen steht vor einem Gebäude.",
    "Ein kleiner Junge spielt mit einem Ball.",
    "Zwei Hunde spielen im Gras.",
    "Die Katze sitzt auf dem Stuhl."
]

model_ready = 'model' in locals() and isinstance(model, Seq2Seq) and hasattr(model, 'encoder')
vocabs_ready = (
    SRC_LANGUAGE in vocab_transform and TRG_LANGUAGE in vocab_transform and
    len(vocab_transform.get(SRC_LANGUAGE, [])) > 4 and 
    len(vocab_transform.get(TRG_LANGUAGE, [])) > 4  
)
tokenizer_ready = SRC_LANGUAGE in token_transform

print(f"Modelo: {type(model).__name__}, Dispositivo: {DEVICE}")
print(f"Vocabulario Fuente ({SRC_LANGUAGE}): {len(vocab_transform[SRC_LANGUAGE])} tokens")
print(f"Vocabulario Objetivo ({TRG_LANGUAGE}): {len(vocab_transform[TRG_LANGUAGE])} tokens")
print("-" * 50)

for i, german_sentence in enumerate(german_sentences_to_translate):
    print(f"\nPrueba {i+1}:")
    print(f"  Oración Fuente (Alemán): {german_sentence}")
    
    english_translation = greedy_search_decode(
        model_to_decode=model,
        src_sentence_str=german_sentence,
        max_len=50 
    )
    
    if english_translation is not None:
        print(f"  Traducción (Inglés)  : {english_translation}")
    else:
        print("  Error: La decodificación no pudo generar una traducción.")
    print("-" * 30)




Modelo: Seq2Seq, Dispositivo: cpu
Vocabulario Fuente (de): 19214 tokens
Vocabulario Objetivo (en): 10837 tokens
--------------------------------------------------

Prueba 1:
  Oración Fuente (Alemán): Ein Mann mit einem roten Helm, der auf einem Pferd reitet.
  Traducción (Inglés)  : A a a a a a a a a a a a a a a .
------------------------------

Prueba 2:
  Oración Fuente (Alemán): Eine Gruppe von Menschen steht vor einem Gebäude.
  Traducción (Inglés)  : A is standing in a of a a of a a of a large . .
------------------------------

Prueba 3:
  Oración Fuente (Alemán): Ein kleiner Junge spielt mit einem Ball.
  Traducción (Inglés)  : This young boy playing a a game of a small small . .
------------------------------

Prueba 4:
  Oración Fuente (Alemán): Zwei Hunde spielen im Gras.
  Traducción (Inglés)  : A soccer players are playing a game in a game of a game .
------------------------------

Prueba 5:
  Oración Fuente (Alemán): Die Katze sitzt auf dem Stuhl.
  Traducción (Inglés)  

In [ ]:
import torch 
import torch.nn.functional as F


def greedy_search_decode_verbose(model_to_decode, src_sentence_str: str, max_len: int = 50,
                                 src_vocab=None, trg_vocab=None, src_tokenizer=None, device_to_use=None,
                                 bos_token_str='<bos>', eos_token_str='<eos>', unk_token_str='<unk>',
                                 top_n_predictions_to_show=5): 
    """
    Genera una traducción usando Greedy Search, mostrando el proceso paso a paso.
    """

    if src_vocab is None: src_vocab = vocab_transform.get(SRC_LANGUAGE)
    if trg_vocab is None: trg_vocab = vocab_transform.get(TRG_LANGUAGE)
    if src_tokenizer is None: src_tokenizer = token_transform.get(SRC_LANGUAGE)
    if device_to_use is None: device_to_use = DEVICE

    if not all([src_vocab, trg_vocab, src_tokenizer, device_to_use, model_to_decode]):
        print("Error en greedy_search_decode_verbose: Faltan componentes esenciales.")
        return None

    model_to_decode.eval()
    print(f"\n--- Decodificando (Greedy Search Detallado) para: '{src_sentence_str}' ---")

    src_tensor = preprocess_sentence(
        src_sentence_str, src_vocab, src_tokenizer, device_to_use,
        bos_token=bos_token_str, eos_token=eos_token_str, unk_token=unk_token_str
    )
    if src_tensor is None: return None

    with torch.no_grad():
        hidden, cell = model_to_decode.encoder(src_tensor)

    try:
        trg_bos_idx = trg_vocab[bos_token_str]
        trg_eos_idx = trg_vocab[eos_token_str]
        itos_map_trg = trg_vocab.get_itos() 
    except KeyError as e:
        print(f"Error: Token especial '{e}' no encontrado en el vocabulario objetivo.")
        return None
        
    trg_indexes = [trg_bos_idx]
    decoded_words = [] 

    print(f"Paso 0: Token inicial de entrada al decodificador: {bos_token_str} (Índice: {trg_bos_idx})")

    for i in range(max_len):
        current_input_token_idx = trg_indexes[-1]
        current_input_token_str = itos_map_trg[current_input_token_idx]
        
        trg_token_tensor = torch.LongTensor([current_input_token_idx]).to(device_to_use)

        with torch.no_grad():
            output_logits, hidden, cell = model_to_decode.decoder(trg_token_tensor, hidden, cell)
        
        output_probs = F.softmax(output_logits, dim=1)
        
        predicted_token_index = output_logits.argmax(1).item()
        predicted_token_str = itos_map_trg[predicted_token_index]
        predicted_token_prob = output_probs[0, predicted_token_index].item()

        print(f"\n  Paso de Decodificación {i+1}:")
        print(f"    Entrada al decodificador: '{current_input_token_str}' (Índice: {current_input_token_idx})")
        
        # Mostrar las N predicciones más probables
        top_k_probs, top_k_indices = torch.topk(output_probs, top_n_predictions_to_show, dim=1)
        print(f"    Predicciones del decodificador (Top {top_n_predictions_to_show}):")
        for k in range(top_n_predictions_to_show):
            token_idx_k = top_k_indices[0, k].item()
            token_str_k = itos_map_trg[token_idx_k]
            token_prob_k = top_k_probs[0, k].item()
            print(f"      - '{token_str_k}' (Índice: {token_idx_k}, Prob: {token_prob_k:.4f})")
        
        print(f"    Token SELECCIONADO (Greedy): '{predicted_token_str}' (Índice: {predicted_token_index}, Prob: {predicted_token_prob:.4f})")

        trg_indexes.append(predicted_token_index)
        
        if predicted_token_index == trg_eos_idx:
            print(f"    Token de Fin de Secuencia ({eos_token_str}) detectado. Finalizando decodificación.")
            break

    final_tokens = []
    for idx in trg_indexes:
        if idx == trg_bos_idx:
            continue # Saltar BOS
        if idx == trg_eos_idx:
            break # Detenerse en EOS
        final_tokens.append(itos_map_trg[idx])
        
    final_translation = " ".join(final_tokens)
    return final_translation



In [14]:

german_sentences_to_translate = [
    "Zwei Hunde spielen im Gras.",
    "Die Katze sitzt auf dem Stuhl."
]

model_ready = 'model' in locals() and isinstance(model, Seq2Seq) and hasattr(model, 'encoder')
vocabs_ready = (
    SRC_LANGUAGE in vocab_transform and TRG_LANGUAGE in vocab_transform and
    len(vocab_transform.get(SRC_LANGUAGE, [])) > 4 and 
    len(vocab_transform.get(TRG_LANGUAGE, [])) > 4  
)
tokenizer_ready = SRC_LANGUAGE in token_transform


print(f"Modelo: {type(model).__name__}, Dispositivo: {DEVICE}")
print(f"Vocabulario Fuente ({SRC_LANGUAGE}): {len(vocab_transform[SRC_LANGUAGE])} tokens")
print(f"Vocabulario Objetivo ({TRG_LANGUAGE}): {len(vocab_transform[TRG_LANGUAGE])} tokens")
print("-" * 50)

for i, german_sentence in enumerate(german_sentences_to_translate):
    print(f"\nPrueba {i+1}:")
    print(f"  Oración Fuente (Alemán): {german_sentence}")
    

    english_translation = greedy_search_decode_verbose(
        model_to_decode=model,
        src_sentence_str=german_sentence,
        max_len=50 
    )
    
    if english_translation is not None:
        print(f"  Traducción (Inglés)  : {english_translation}")
    else:
        print("  Error: La decodificación no pudo generar una traducción.")
    print("-" * 30)



Modelo: Seq2Seq, Dispositivo: cpu
Vocabulario Fuente (de): 19214 tokens
Vocabulario Objetivo (en): 10837 tokens
--------------------------------------------------

Prueba 1:
  Oración Fuente (Alemán): Zwei Hunde spielen im Gras.

--- Decodificando (Greedy Search Detallado) para: 'Zwei Hunde spielen im Gras.' ---
Paso 0: Token inicial de entrada al decodificador: <bos> (Índice: 2)

  Paso de Decodificación 1:
    Entrada al decodificador: '<bos>' (Índice: 2)
    Predicciones del decodificador (Top 5):
      - 'A' (Índice: 6, Prob: 0.2273)
      - 'Three' (Índice: 59, Prob: 0.1925)
      - 'There' (Índice: 223, Prob: 0.1497)
      - 'Many' (Índice: 381, Prob: 0.0898)
      - 'Several' (Índice: 165, Prob: 0.0795)
    Token SELECCIONADO (Greedy): 'A' (Índice: 6, Prob: 0.2273)

  Paso de Decodificación 2:
    Entrada al decodificador: 'A' (Índice: 6)
    Predicciones del decodificador (Top 5):
      - 'soccer' (Índice: 129, Prob: 0.2084)
      - 'hockey' (Índice: 377, Prob: 0.0626)
      - 